In [45]:
from qiskit import *
from qiskit.visualization import plot_histogram, plot_bloch_multivector
from qiskit_aer import Aer

### Classical Search

In [2]:
my_list = [5,4,6,9,1,2,3,7,8,0]

In [3]:
def oracle(number):
    winning_number = 8

    if number == winning_number:
        response = True
    else:
        response = False

    return response

In [ ]:
for index, number in enumerate(my_list):
    if oracle(number) is True:
        print(f"Winning number index: {index}")
        print(f"Execution count: {index + 1}")
        break

### Quantum Model Grover's Search Algorithm

In [ ]:
# oracle circuit - (winning state 00)+
oracle_circuit = QuantumCircuit(2, name="Oracle Circuit")
oracle_circuit.x([0,1])
oracle_circuit.cz([0], [1])
oracle_circuit.x([0,1])
oracle_circuit.to_gate() #  creates circuit as a gate you can apply to another ciruit
oracle_circuit.draw(output="mpl")

In [ ]:
# oracle circuit - (winning state 11)+

oracle_circuit = QuantumCircuit(2, name="Oracle Circuit")
oracle_circuit.cz(0,1)
oracle_circuit.to_gate() #  creates circuit as a gate you can apply to another ciruit
oracle_circuit.draw(output="mpl")

In [ ]:
qreg_q = QuantumRegister(2, 'q')
creg_c = ClassicalRegister(2, 'c')
main_circuit = QuantumCircuit(qreg_q, creg_c)

main_circuit.h(qreg_q[:])
main_circuit.append(oracle_circuit, qreg_q[:])

In [ ]:
main_circuit.draw(output="mpl")

In [ ]:
relflection_circuit = QuantumCircuit(2, name="Relflection Circuit")

relflection_circuit.h([0,1])
relflection_circuit.z([0,1])
relflection_circuit.cz(0,1)
relflection_circuit.h([0,1])
relflection_circuit.to_gate()

relflection_circuit.draw(output="mpl")

In [ ]:
main_circuit.append(relflection_circuit, [0,1])
main_circuit.measure([0,1],[0,1])
main_circuit.draw(output="mpl")

### Simulate Results and Statevector

In [40]:
qasm_simulator = Aer.get_backend('qasm_simulator')
statevector_simulator = Aer.get_backend('statevector_simulator')

In [41]:
def run_on_simulators(circuit, shots):
    new_circuit = transpile(circuit, statevector_simulator)
    statevector_job = statevector_simulator.run(new_circuit, backend=qasm_simulator)
    result = statevector_job.result()
    statevector = result.get_statevector()

    num_qubits = circuit.num_qubits
    circuit.measure([i for i in range(num_qubits)], [i for i in range(num_qubits)])

    new_circuit = transpile(circuit, qasm_simulator)
    qasm_job = qasm_simulator.run(new_circuit, backend=qasm_simulator, shots=shots).result()
    counts = qasm_job.get_counts()
    
    return statevector, counts

In [42]:
statevector, counts = run_on_simulators(main_circuit, 1024)

In [ ]:
plot_histogram([counts])

In [ ]:
plot_bloch_multivector(statevector)

### Real Quantum Computer

In [15]:
from qiskit_ibm_runtime import QiskitRuntimeService

In [ ]:
provider = QiskitRuntimeService(channel="ibm_quantum")
provider.backends()

In [ ]:
for backend in provider.backends():
    try:
        qubit_count = len(backend.properties().qubits)
    except:
        qubit_count = "simulated"

    print(f"{backend.name} : {backend.status().pending_jobs} & {qubit_count} qubits")

In [18]:
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler

In [19]:
provider = QiskitRuntimeService()
quantum_computer = provider.backend("ibm_kyiv")
pm = generate_preset_pass_manager(backend=quantum_computer, optimization_level=1)
isa_circuit = pm.run(main_circuit)
sampler = Sampler(quantum_computer)
job = sampler.run([isa_circuit])

In [ ]:
job.status()

In [ ]:
result = job.result()
plot_histogram([result[0].data.c.get_counts()])